In [ ]:
from msal import PublicClientApplication
import os
import webbrowser

# Cargar configuraciones desde variables de entorno (puedes usar dotenv si prefieres)
TENANT_ID = "e994072b-523e-4bfe-86e2-442c5e10b244"
CLIENT_ID = "d4f0a82a-0933-4dad-b584-1fa5cd7ab1e3"
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["Files.Read.All"]  # Permiso para leer archivos en SharePoint

# Crear aplicación MSAL
app = PublicClientApplication(
    client_id=CLIENT_ID,
    authority=AUTHORITY
)
# Intentar obtener un token silenciosamente desde la caché
accounts = app.get_accounts()
if accounts:
    result = app.acquire_token_silent(SCOPE, account=accounts[0])
else:
    # Solicitar interactivamente el login (dispara el navegador)
    flow = app.initiate_device_flow(scopes=SCOPE)
    if "user_code" not in flow:
        raise Exception("No se pudo iniciar el flujo de dispositivo")
    print("Por favor, ve a:", flow["verification_uri"])
    print("e ingresa el código:", flow["user_code"])
    result = app.acquire_token_by_device_flow(flow)

# Verificación del resultado
if "access_token" in result:
    print("✅ Autenticación exitosa!")
    access_token = result["access_token"]
    os.environ["ACCESS_TOKEN"] = access_token
else:
    print("❌ Error en la autenticación:", result.get("error_description"))